# Read and write Raster Files using GDAL

In [ ]:
"""
Send Array to Raster
"""

def obj_to_rst(inArray, outRst, template, noData=None, geotrans=None):
    from osgeo           import gdal, osr, gdal_array
    from pycode.geofiles import drv_name

    if type(template).__name__ == 'Dataset':
        img_template = template
    else:
        img_template  = gdal.Open(template)
        
    geo_transform = img_template.GetGeoTransform() if not geotrans else \
        geotrans
    rows, cols    = inArray.shape
    drv_n         = drv_name(outRst)
    driver        = gdal.GetDriverByName(drv_n)
    
    out       = driver.Create(
        outRst, cols, rows, 1,
        gdal_array.NumericTypeCodeToGDALTypeCode(inArray.dtype)
    )
    
    out.SetGeoTransform(geo_transform)
    outBand       = out.GetRasterBand(1)
    
    if noData or noData == 0:
        outBand.SetNoDataValue(noData)
        
    outBand.WriteArray(inArray)
        
    proj = osr.SpatialReference(wkt=img_template.GetProjection())
    
    if proj:
        out.SetProjection(img_template.GetProjection())
    
    outBand.FlushCache()
    
    return outRst

In [ ]:
"""
Apply Normalized Difference NIR/Red Normalized Difference
Vegetation Index, Calibrated NDVI - CDVI
    
https://www.indexdatabase.de/db/i-single.php?id=58
    
EXPRESSION: (nir - red) / (nir + red)
"""

def ndvi(nir, red, outRst):
    
    
    import numpy as np
    from osgeo   import gdal, gdal_array
    
    # Open Images
    src_nir = gdal.Open(nir, gdal.GA_ReadOnly)
    src_red = gdal.Open(red, gdal.GA_ReadOnly)
    
    # To Array
    num_nir = src_nir.GetRasterBand(1).ReadAsArray().astype(float)
    num_red = src_red.GetRasterBand(1).ReadAsArray().astype(float)
    
    # Do Calculation
    ndvi = (num_nir - num_red) / (num_nir + num_red)
    
    # Place NoData Value
    nirNdVal = src_nir.GetRasterBand(1).GetNoDataValue()
    redNdVal = src_red.GetRasterBand(1).GetNoDataValue()
    
    ndNdvi = np.amin(ndvi) - 1
    
    np.place(ndvi, num_nir==nirNdVal, ndNdvi)
    np.place(ndvi, num_red==redNdVal, ndNdvi)
    
    # Export Result
    return obj_to_rst(ndvi, outRst, nir, noData=ndNdvi)

red = '../geodata/mod1demo4/T29TNE_20200703T112119_B04_10m.tif'
nir = '../geodata/mod1demo4/T29TNE_20200703T112119_B08_10m.tif'
out = '../geodata/mod1demo4/ndvi_tst.tif'

ndvi(nir, red, out)